In [2]:
using BenchmarkTools

In [1]:
function find_sum_of_sqrt_vectors()
    nvectors =10000
    sumvector = Vector{Float64}(undef,nvectors)
    for i = 1:nvectors
        # v = sqrt.(1:i)
        sumvector[i] = sum(sqrt.(1:i))
    end
end
@profview find_sum_of_sqrt_vectors();

In [10]:
function find_sum_of_sqrt_vectors_reusemem()
    nvectors = 10000
    sumvector = Vector{Float64}(undef,nvectors)
    v = Vector{Float64}(undef,nvectors)
    for i = 1:nvectors
        v[1:i] .= sqrt.(1:i)
        sumvector[i] = sum(v)
        v .= 0
    end
end
@btime find_sum_of_sqrt_vectors_reusemem();


  105.021 ms (4 allocations: 156.34 KiB)


In [12]:
function vvv_add()
    L = 2; M = 100; N = 5
    vec_vec_vec = [[rand(N) for j in 1:M] for i in 1:L];
    vec_mat = [rand(M, N) for i in 1:L];
    arr = rand(L, M, N);
    for i = 1:N
        for j = 1:M
            for k = 1:L
                vec_vec_vec[k][j][i] += 1
            end
        end
    end
end

function vmat_add()
    L = 2; M = 100; N = 5
    vec_vec_vec = [[rand(N) for j in 1:M] for i in 1:L];
    vec_mat = [rand(M, N) for i in 1:L];
    arr = rand(L, M, N);
    for i = 1:N
        for j = 1:M
            for k = 1:L
                vec_mat[k][j,i] += 1
            end
        end
    end
end

function arr_add()
    L = 2; M = 100; N = 5
    vec_vec_vec = [[rand(N) for j in 1:M] for i in 1:L];
    vec_mat = [rand(M, N) for i in 1:L];
    arr = rand(L, M, N);
    for i = 1:N
        for j = 1:M
            for k = 1:L
                arr[k,j,i] += 1
            end
        end
    end
end

@btime vvv_add()
@btime vmat_add()
@btime arr_add()


  8.218 μs (207 allocations: 36.69 KiB)
  7.578 μs (207 allocations: 36.69 KiB)
  7.089 μs (207 allocations: 36.69 KiB)


In [2]:
using BenchmarkTools
L=M=N=100
arr = rand(L, M, N)
function arr_1(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N
        for j = 1:M
            for k = 1:L
                dummy[k, j, i] = arr[k,j,i] 
            end
        end
    end
end
function arr_2(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N
        for j = 1:M
            dummy[:, j, i] = arr[:,j,i] 
        end
    end
end
function arr_3(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N
        dummy[:, :, i] = arr[:,:,i] 
    end
end
@btime arr_1(arr, L, M, N)
@btime arr_2(arr, L, M, N)
@btime arr_3(arr, L, M, N)

  2.047 ms (2 allocations: 7.63 MiB)
  2.293 ms (10002 allocations: 16.17 MiB)
  2.543 ms (202 allocations: 15.26 MiB)


In [ ]:
L=M=N=100
arr = rand(L, M, N);
function arr_1(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N ;for j = 1:M; for k = 1:L; dummy[k, j, i] = arr[k,j,i] 
    end; end ;end
end
function arr_2(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N
        for j = 1:M
            dummy[:, j, i] = arr[:,j,i] 
        end
    end
end
function arr_3(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N ; for k = 1:L; dummy[k, :, i] = arr[k,:,i] 
    end; end
end
function arr_4(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for j = 1:M; for k = 1:L; dummy[k, j, :] = arr[k,j,:] 
    end; end
end
function arr_5(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for i = 1:N
        dummy[:, :, i] = arr[:,:,i] 
    end
end
function arr_6(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for j = 1:M
        dummy[:, j, :] = arr[:,j,:] 
    end
end
function arr_7(arr, L::Int64, M::Int64, N::Int64)
    dummy = zeros(L, M, N)
    for k = 1:L
        dummy[k, :, :] = arr[k,:,:] 
    end
end

@btime arr_1(arr, L, M, N)
@btime arr_2(arr, L, M, N)
@btime arr_3(arr, L, M, N)
@btime arr_4(arr, L, M, N)
@btime arr_5(arr, L, M, N)
@btime arr_6(arr, L, M, N)
@btime arr_7(arr, L, M, N)

In [3]:
using BenchmarkTools

In [8]:
function main()
    function biss!(A, i=1, j=length(A))
        m = round(Int, (i + j) / 2)
        A[m] = 1
        if j == i+2; return; end
        biss!(A, i, m)
        biss!(A, m, j)
    end
    A = zeros(9)
    biss!(A)
    # println(A)
end
@btime main()

  216.996 ns (3 allocations: 192 bytes)


In [9]:
function main()
function biss!(A, i=1, j=length(A))
    m = round(Int, (i + j) / 2)
    A[m] = 1
    if j == i+2; return; end
    biss!(A, i, m)
    biss!(A, m, j)
end
function init()
    A = zeros(9)
    biss!(A)
    # println(A)
end
init()
end
@btime main()

  218.563 ns (3 allocations: 192 bytes)


In [1]:
using BenchmarkTools

In [2]:
using BenchmarkTools
struct Alphabet
    a::Int64
    b::Int64
    c::Int64
    d::Int64
    e::Int64
    f::Int64
    g::Int64
    h::Int64
    i::Int64
    k::Int64
    l::Int64
end
function mainst()
    function st(a::Alphabet)
        s = a.a*a.b*a.c
        for i in 1:1
        arr = rand(s)
        end
    end
    alpha = Alphabet(1,2,3,4,5,6,7,8,9,10,11)
    st(alpha) 
end

function mainvar()
    function var(a::Int64, b::Int64, c::Int64)
        s = a*b*c
        for i in 1:1
        arr = rand(s)
        end
    end
    alpha = Alphabet(1,2,3,4,5,6,7,8,9,10,11)
    var(alpha.a, alpha.b, alpha.c)
end

@btime mainst()
@btime mainvar()


  28.428 ns (1 allocation: 112 bytes)
  28.989 ns (1 allocation: 112 bytes)


In [ ]:

using BenchmarkTools, StaticArrays

function test0()
    items = @SVector [i for i in 1:10000]
    a =[]
    for i in 1:n
        push!(a, items[i]) 
    end
end

function test1(n)
    r = [i for i in 1:n]
    mySVector = SVector{n}
    items = mySVector(r)
    a =[]
    for i in 1:n
        push!(a, items[i]) 
    end
end

function test2(n)
    items = SVector{n}([i for i in 1:n])
    a =[]
    for i in 1:n
        push!(a, items[i]) 
    end
end

function test3(n)
    items = [i for i in 1:n]
    a =[]
    for i in 1:n
        push!(a, items[i]) 
    end
end
n=10000
@btime test0()
@btime test1($n)
@btime test2($n)
@btime test3($n)


In [30]:
using BenchmarkTools
function colfirst()
    r = rand(10^4,10^4)
    dump = rand(10^4,10^4)
    for i in size(r, 1), j in size(r, 2)
        dump[i, j] = r[i, j]
    end
end
@btime colfirst()
function rowfirst()
    r = rand(10^4,10^4)
    dump = rand(10^4,10^4)
    for j in size(r, 2), i in size(r, 1)
        dump[i, j] = r[i, j]
    end
end
@btime rowfirst()



  261.614 ms (4 allocations: 1.49 GiB)
  261.345 ms (4 allocations: 1.49 GiB)


In [2]:
using BenchmarkTools

In [3]:
function copy_cols(x::Vector{T}) where T
    inds = axes(x, 1)
    out = similar(Array{T}, inds, inds)
    for i = inds
        out[:, i] = x
    end
    return out
end

function copy_rows(x::Vector{T}) where T
    inds = axes(x, 1)
    out = similar(Array{T}, inds, inds)
    for i = inds
        out[i, :] = x
    end
    return out
end

function copy_col_row(x::Vector{T}) where T
    inds = axes(x, 1)
    out = similar(Array{T}, inds, inds)
    for col = inds, row = inds
        out[row, col] = x[row]
    end
    return out
end

function copy_row_col(x::Vector{T}) where T
    inds = axes(x, 1)
    out = similar(Array{T}, inds, inds)
    for row = inds, col = inds
        out[row, col] = x[col]
    end
    return out
end
const x = randn(10000);
@btime copy_cols($x);
@btime copy_rows($x);
@btime copy_col_row($x);
@btime copy_row_col($x);

  134.588 ms (2 allocations: 762.94 MiB)
  439.710 ms (2 allocations: 762.94 MiB)
  148.438 ms (2 allocations: 762.94 MiB)
  491.200 ms (2 allocations: 762.94 MiB)


In [3]:
using BenchmarkTools
function ct()
    x=rand([missing, 5.0],10^6)
    return count(.!isa.(x, Missing))
end

function forloop()
    x=rand([missing, 5.0],10^6)
    s=0
    for _ in skipmissing(x)
        s += 1
    end
    return s
end
@btime ct()
@btime forloop()

500282
500044
  10.071 ms (10 allocations: 8.71 MiB)
  13.083 ms (6 allocations: 8.58 MiB)


499021

In [2]:
using BenchmarkTools

In [12]:
using BenchmarkTools
function forrand()
    x = zeros(2, 10000);
    for j in 1:size(x, 2), i in 1:size(x, 1)
        x[i, j] = 2 * (rand()-0.5)
    end
end
function broadrand()
    r = 2 .* (rand(Float64, (2, 10000)) .- 0.5)
end
function eachrand()
    x = zeros(2, 10000);
    for i in eachindex(x)
        x[i] = 2 * (rand()-0.5)
    end
end
function forrand1()
    x = zeros(10000, 2);
    for j in 1:size(x, 2), i in 1:size(x, 1)
        x[i, j] = 2 * (rand()-0.5)
    end
end
function broadrand1()
    r = 2 .* (rand(Float64, (10000, 2)) .- 0.5)
end
function eachrand1()
    x = zeros(10000, 2);
    for i in eachindex(x)
        x[i] = 2 * (rand()-0.5)
    end
end
@btime forrand()
@btime broadrand()
@btime eachrand()
@btime forrand1()
@btime broadrand1()
@btime eachrand1()

  31.149 μs (2 allocations: 156.30 KiB)
  30.241 μs (4 allocations: 312.59 KiB)
  28.914 μs (2 allocations: 156.30 KiB)
  29.403 μs (2 allocations: 156.30 KiB)
  21.791 μs (4 allocations: 312.59 KiB)
  29.752 μs (2 allocations: 156.30 KiB)


In [11]:
for i in 5
    println(i)
end

5


In [9]:
using BenchmarkTools
function atbroad()
    M=1000;dim=2; Ncycle=60
    x = rand(M, dim, Ncycle)
    y = zeros(M, dim, Ncycle)
    for i in 1:Ncycle
        @. y =  x[:, :, i]
    end
end
function forloop()
    M=1000;dim=2; Ncycle=60
    x = rand(M, dim, Ncycle)
    y = zeros(M, dim, Ncycle)
    for i in 1:Ncycle, d in 1:dim, m in 1:M
        y[m, d, i] = x[m, d, i]
    end
end
function rdm()
    M=1000;dim=2; Ncycle=60
    x = rand(M, dim, Ncycle)
    y = zeros(M, dim, Ncycle)
    for i in 1:Ncycle
        y[:, :, i] =  x[:, :, i]
    end
end
@btime atbroad()
@btime forloop()
@btime rdm()

  895.155 μs (64 allocations: 2.75 MiB)
  219.512 μs (4 allocations: 1.83 MiB)
  245.073 μs (64 allocations: 2.75 MiB)


In [25]:
using BenchmarkTools
function distance1(x1::Float64, x2::Float64, boundary::Float64)::Float64
    dx = abs(x1 - x2)
    min(dx, 2 * boundary - dx)
end
function distance2(x1::Float64, x2::Float64, boundary::Float64)::Float64
    dx = abs(x1 - x2)
    dx > boundary ? 2 * boundary - dx : dx
end
x = rand(10_000_000)
y = rand(10_000_000)
@btime distance1.($x, $y, 0.5)
@btime distance2.($x, $y, 0.5)

  14.658 ms (2 allocations: 76.29 MiB)
  15.492 ms (2 allocations: 76.29 MiB)


10000000-element Vector{Float64}:
 0.4049136212191221
 0.21980237049623286
 0.22489123225786023
 0.48793014045907235
 0.38249667334064774
 0.4881100066200442
 0.48950334396183925
 0.1315845923349087
 0.31368210156049503
 0.2949851039694452
 ⋮
 0.04762929665692872
 0.2557140977409813
 0.24832377413196216
 0.05323163843077072
 0.33445142347814005
 0.14164887188560504
 0.23496866855971477
 0.06470253817545368
 0.09423270975539633

In [14]:
x = [rand(2) for i in 1:1000000]

1000000-element Vector{Vector{Float64}}:
 [0.4698331419368619, 0.5552780356035049]
 [0.10883441986430553, 0.8628984247793724]
 [0.983490320348385, 0.29393054516124306]
 [0.04462508874427451, 0.7458328059458946]
 [0.7041990812536707, 0.7656729968838429]
 [0.34835068981962336, 0.8688641293524746]
 [0.9896981126817183, 0.43670732684323244]
 [0.9270021530252534, 0.282050177025701]
 [0.3036579012031868, 0.20526942621319788]
 [0.48381198144042326, 0.950699133748487]
 ⋮
 [0.3894523477141233, 0.4547198101783525]
 [0.9086674956598934, 0.7529854071716681]
 [0.37429836911513625, 0.15535423258221925]
 [0.003442799330951818, 0.6784578696504263]
 [0.936532179024221, 0.7703286207982324]
 [0.6016027314461229, 0.4595325595610328]
 [0.8151792493326662, 0.5105872464292461]
 [0.817339432923094, 0.9498870536338565]
 [0.7658269959968301, 0.9512828663721856]

In [9]:
using StaticArrays, BenchmarkTools, Setfield
function copySized()
    v = SizedVector{10, Float64}(zeros(10))
    w = Vector{Float64}(undef, 10*2)
    for i in eachindex(v)
        v[i] = rand()
    end
    for i in eachindex(v)
        j = i+floor(Int64, 10/4)
        w[j] = v[i]
    end
end
function copyStatic()
    v = @SVector zeros(10)
    w = Vector{Int64}(undef, 10*2)
    for i in eachindex(v)
       @set v[i] = rand()
    end
    for i in eachindex(v)
        j = i+floor(Int64, 10/4)
        w[j] = v[i]
    end
end
function copynormal()
    v = zeros(10)
    w = Vector{Float64}(undef, 10*2)
    for i in eachindex(v)
        v[i] = rand()
    end
    for i in eachindex(v)
        j = i+floor(Int64, 10/4)
        w[j] = v[i]
    end
end

@btime copySized()
@btime copyStatic()
@btime copynormal()

  110.162 ns (3 allocations: 512 bytes)
  48.133 ns (1 allocation: 224 bytes)
  92.045 ns (2 allocations: 368 bytes)


In [4]:
using BenchmarkTools, StaticArrays
function f(x)
    z = 0.
    for i in 1:10
      y = [ i+1, i+2 ]
      z += x[1]*y[1] + x[2]*y[2]
    end
  end
 function g(x)
    z = 0.
    for i in 1:10
      y = @SVector [ i+1, i+2 ]
      z += x[1]*y[1] + x[2]*y[2]
    end
  end
  function h(x)
    z = 0.
    for i in 1:10
      y = (i+1, i+2)
      z += x[1]*y[1] + x[2]*y[2]
    end
  end
@btime f(x) setup=(x=rand(2)) 
@btime g(x) setup=(x=rand(2)) 
@btime h(x) setup=(x=rand(2)) 

  255.249 ns (10 allocations: 800 bytes)
  1.300 ns (0 allocations: 0 bytes)
  1.500 ns (0 allocations: 0 bytes)


In [9]:
using  BenchmarkTools
function f(n::Int64)
    x = rand(n)
    s = sum(x)
end
function g(n::Int64)
    x = rand(n)
    s = sum(view(x, :))
end
function h(n::Int64)
    x = rand(n)
    s = 0
    for i in x
        s += i
    end
end
@btime f(n) setup=(n=10) 
@btime g(n) setup=(n=10) 
@btime h(n) setup=(n=10) 

  102.225 ns (1 allocation: 144 bytes)
  103.936 ns (1 allocation: 144 bytes)
  99.683 ns (1 allocation: 144 bytes)


In [12]:
using  BenchmarkTools
function f(n::Int64)
    collect(1:n)
end
function g(n::Int64)
    [i for i in 1:n]
end
@btime f(n) setup=(n=10000) 
@btime g(n) setup=(n=10000) 

  5.225 μs (2 allocations: 78.17 KiB)
  5.680 μs (2 allocations: 78.17 KiB)


10000-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
     ⋮
  9992
  9993
  9994
  9995
  9996
  9997
  9998
  9999
 10000

In [2]:
using BenchmarkTools
function miscollect()
    v = rand([1,2,3,4,5,6,7,8,9,10,missing], 1000000)
    return collect(skipmissing(v))
end
function misfilter()
    v = rand([1,2,3,4,5,6,7,8,9,10,missing], 1000000)
    return filter!(!ismissing, v)
end

@btime miscollect()
@btime misfilter()


  13.220 ms (28 allocations: 18.37 MiB)
  8.122 ms (14 allocations: 8.58 MiB)


909116-element Vector{Union{Missing, Int64}}:
 10
  8
  2
  9
  6
  4
  8
  3
  7
  7
  ⋮
  8
  4
 10
  5
  7
  9
  7
  4
  6

In [9]:
using BenchmarkTools
f(x::Float64, y::Float64)=log(exp(x) * y)
g(x::Float64, y::Float64)=x + log(y)
x=5.3
y=6.3
@btime  f($x, $y)
@btime  g($x, $y)

  10.767 ns (0 allocations: 0 bytes)
  5.168 ns (0 allocations: 0 bytes)


7.1405496333974865

In [6]:
using BenchmarkTools
function test1()
    r = rand(2,1000)
    tmp = zeros(2)
    for i in size(r, 2)
        tmp = r[:, i]
    end
end
function test2()
    r = rand(2,1000)
    tmp = zeros(2)
    for i in size(r, 2)
        for j in size(r, 1)
            tmp = r[j, i]
        end
    end
end
function main()
    @btime test1()
    @btime test2() # faster
end
main()


true

In [ ]:
using BenchmarkTools
function fx()
    x = rand(10^6)
    3x.^2 + 4x + 7x.^3
end
function fdotx()
    x = rand(10^6)
    @. 3x^2 + 4x + 7x^3
end
function f(x::Vector{Float64})
    3x.^2 + 4x + 7x.^3
end
function fdot(x::Vector{Float64})
    @. 3x^2 + 4x + 7x^3
end
function f(x::Float64)
    3x^2 + 4x + 7x^3
end

const x = rand(10^6);
@btime fx()
@btime fdotx()
@btime f(x)
@btime fdot(x)
@btime f.(x)

In [ ]:
using BenchmarkTools
function xinc(x)
    return [x, x+1, x+2]
end;

function loopinc()
    y = 0
    for i = 1:10^7
        ret = xinc(i)
        y += ret[2]
    end
    return y
end;

function xinc!(ret::AbstractVector{T}, x::T) where T
    ret[1] = x
    ret[2] = x+1
    ret[3] = x+2
    nothing
end;

function loopinc_prealloc()
    ret = Vector{Int}(undef, 3)
    y = 0
    for i = 1:10^7
        xinc!(ret, i)
        y += ret[2]
    end
    return y
end;

@btime loopinc()
@btime loopinc_prealloc()

In [13]:
using BenchmarkTools
function copyview(a, b)
    for j in eachindex(view(a, :, 1))
        b[j, :] = @view a[j, :]
    end
end
function copyview′(a, b)
    for j in eachindex(a)
        b[j] = a[j]
    end
end
function copy1(a, b)
    for j in eachindex(view(a, :, 1))
        b[j, 1] = a[j, 1]
    end
end

function copynormal(a, b)
    for j in eachindex(view(a, :, 1))
        b[j, :] =  a[j, :]
    end
end

function copycopy(a, b)
    for j in eachindex(view(a, :, 1))
        b[j, :] = copy(a[j, :]) 
    end
end
a = ones(100, 2)
b = zeros(100, 2)
@btime copyview($a, $b)
@btime copyview′($a, $b)
@btime copy1($a, $b)
@btime copynormal($a, $b)
@btime copycopy($a, $b)


  914.442 ns (0 allocations: 0 bytes)
  49.745 ns (0 allocations: 0 bytes)
  49.745 ns (0 allocations: 0 bytes)
  2.219 μs (100 allocations: 7.81 KiB)
  3.841 μs (200 allocations: 15.62 KiB)


In [11]:
@btime copyview($a, $b)
b

  926.811 ns (0 allocations: 0 bytes)


100×2 Matrix{Float64}:
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 ⋮    
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0

In [15]:
using LinearAlgebra
function dfunc_all(i,j,x,y)
    (x[1,i]-y[1,j])^2 + (x[2,i]-y[2,j])^2 + (x[3,i]-y[3,j])^2
end

function dfunc_subarray(x,y)
    (x[1]-y[1])^2 + (x[2]-y[2])^2 + (x[3]-y[3])^2
end

function dfunc_view(x,y)
    (x[1]-y[1])^2 + (x[2]-y[2])^2 + (x[3]-y[3])^2
end

function dist(n,x,y,typ)
    acc = zero(eltype(x))
    for i in 1:n
        for j in 1:n
            if typ == 1
                d = dfunc_all(i,j,x,y)
            end
            if typ == 2
                d = dfunc_subarray(x[:,i],y[:,j])
            end
            if typ == 3
                d = dfunc_view(view(x,:,i),view(y,:,j))
            end
            if typ == 4
                d = dot(view(x,:,i),view(y,:,j))
            end

            acc += d
        end
    end
    acc
end

n = 10000;
x = rand(3,n);
y = rand(3,n);

using BenchmarkTools
print("dfunc_all: ")
@btime dist($n,$x,$y,1)
print("dfunc_subarray: ")
@btime dist($n,$x,$y,2)
print("dfunc_view: ")
@btime dist($n,$x,$y,3)
print("dot_view: ")
@btime dist($n,$x,$y,4)

dfunc_all:   172.881 ms (0 allocations: 0 bytes)
dfunc_subarray:   3.976 s (200000000 allocations: 14.90 GiB)
dfunc_view:   216.144 ms (0 allocations: 0 bytes)
dot_view:   9.004 s (0 allocations: 0 bytes)


7.489174706789348e7

In [2]:
using BenchmarkTools
function forloop(a, b)
    for j in eachindex(a)
        a[j] = 1
        b[j] = 2
    end
end
function forforloop(a, b)
    for j in eachindex(a)
        a[j] = 1
    end
    for j in eachindex(a)
        b[j] = 2
    end
end
a = zeros(100)
b = zeros(100)
@btime forloop($a, $b)
@btime forforloop($a, $b)

  65.535 ns (0 allocations: 0 bytes)
  125.112 ns (0 allocations: 0 bytes)
